<a href="https://colab.research.google.com/github/KUSUMAJOGARAO/Edunet/blob/main/MLProject_Bank.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Import necessary libraries:**

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import numpy as np
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report
from sklearn.feature_selection import chi2


---


**Read excel file:**

In [2]:
url = 'https://raw.githubusercontent.com/dzubii/MLProject/master/bank.csv'
df = pd.read_csv(url, delimiter = ';')

one_hot_encoder = OneHotEncoder()
one_hot_encoder.fit(df.drop(["y"], axis=1))
X = one_hot_encoder.transform(df.drop(["y"], axis=1))

label_encoder = LabelEncoder()
label_encoder.fit(df["y"])
y = label_encoder.transform(df["y"])



---


**Show data info:**

In [3]:
df.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,30,unemployed,married,primary,no,1787,no,no,cellular,19,oct,79,1,-1,0,unknown,no
1,33,services,married,secondary,no,4789,yes,yes,cellular,11,may,220,1,339,4,failure,no
2,35,management,single,tertiary,no,1350,yes,no,cellular,16,apr,185,1,330,1,failure,no
3,30,management,married,tertiary,no,1476,yes,yes,unknown,3,jun,199,4,-1,0,unknown,no
4,59,blue-collar,married,secondary,no,0,yes,no,unknown,5,may,226,1,-1,0,unknown,no


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4521 entries, 0 to 4520
Data columns (total 17 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   age        4521 non-null   int64 
 1   job        4521 non-null   object
 2   marital    4521 non-null   object
 3   education  4521 non-null   object
 4   default    4521 non-null   object
 5   balance    4521 non-null   int64 
 6   housing    4521 non-null   object
 7   loan       4521 non-null   object
 8   contact    4521 non-null   object
 9   day        4521 non-null   int64 
 10  month      4521 non-null   object
 11  duration   4521 non-null   int64 
 12  campaign   4521 non-null   int64 
 13  pdays      4521 non-null   int64 
 14  previous   4521 non-null   int64 
 15  poutcome   4521 non-null   object
 16  y          4521 non-null   object
dtypes: int64(7), object(10)
memory usage: 600.6+ KB


In [5]:
df.var()

<ipython-input-5-28ded241fd7c>:1: FutureWarning: The default value of numeric_only in DataFrame.var is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  df.var()


age         1.118562e+02
balance     9.057922e+06
day         6.802402e+01
duration    6.752547e+04
campaign    9.670897e+00
pdays       1.002424e+04
previous    2.868153e+00
dtype: float64



---

**Feature Selection:**

In [6]:
df = pd.DataFrame({
    'Feature1': [1, 2, 3, 4],
    'Feature2': [5, 6, 7, 8],
    'y': [0, 1, 0, 1]
})

features = df.drop(["y"], axis=1)  # Assuming "y" is the target variable
labels = df["y"]

chivalues = chi2(features, labels)[0]
chi = pd.DataFrame(list(zip(df.drop(["y"], axis=1).columns, chivalues)), columns=["Features", "chi2"])



---


**Creating pipeline and fitting to training data:**

In [7]:
# Pipeline

scaler = StandardScaler(with_mean=False)
X = scaler.fit_transform(X)

# Setting up the pipeline
#steps = [('scaler', StandardScaler()), ('SVM', SVC())]
#steps = ('SVM', SVC())
#pipeline = Pipeline(steps)

# Specifying the hyperparameter space
parameters = {'C':[0.1, 1, 5, 10], 'gamma':[0.0001, 0.001, 0.01, 0.1, 0.2, 0.4, 0.8]}

# Creating train and test sets
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3, random_state = 42)

# Instantiating GridSearchCV and fitting to the training set
cv = GridSearchCV(SVC(), param_grid=parameters, cv=5)
cv.fit(X_train,y_train)


# Predicting the labels of the test set: y_pred
y_pred = cv.predict(X_test)

# Computing metrics
print("Accuracy: {}".format(cv.score(X_test, y_test)))
print(classification_report(y_test, y_pred))
print("Tuned Model Parameters: {}".format(cv.best_params_))

Accuracy: 0.887988209285188
              precision    recall  f1-score   support

           0       0.89      1.00      0.94      1205
           1       0.00      0.00      0.00       152

    accuracy                           0.89      1357
   macro avg       0.44      0.50      0.47      1357
weighted avg       0.79      0.89      0.84      1357

Tuned Model Parameters: {'C': 0.1, 'gamma': 0.0001}


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
